# Evaluate finetuned Mistral Models

## Imports and environment variables

In [33]:
import torch
import os
import sys
from dotenv import load_dotenv
import ast
import evaluate

from peft import PeftModel

import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, AutoModelForCausalLM

# Load environment variables from .env file
load_dotenv()

# Add the parent directory to the Python path
# __file__


# Adjust this path to point to the directory containing rl_training_new
module_path = os.path.abspath(os.path.join('..')) # or another relative path
if module_path not in sys.path:
    sys.path.append(module_path)

from rl_training_new.utils import find_best_window


from bert_score import score

# load the relevant devices available on the server
os.environ["CUDA_VISIBLE_DEVICES"] = os.getenv("AVAILABLE_DEVICES")

# Enable expandable CUDA segments
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# load cuda
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print("CUDA is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")


There are 1 GPU(s) available.
CUDA is available. Using GPU: NVIDIA L40S


In [2]:
MODEL = os.getenv("GENERATION_MODEL_NAME")
ALGORITHM = os.getenv("EVAL_MODEL_ALGORITHM")
RL_TRAINED_ADAPTERS = os.getenv("EVAL_MODEL_FOLDER")
EVAL_ANSWERS_CSV = os.getenv("EVAL_ANSWERS_CSV")

## Load model

In [3]:
base_model = AutoModelForCausalLM.from_pretrained(MODEL)
new_model = PeftModel.from_pretrained(base_model, RL_TRAINED_ADAPTERS)

tokenizer = AutoTokenizer.from_pretrained(MODEL)

base_model.eval()
new_model.eval()

base_model.to(device)
new_model.to(device)


Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.78s/it]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(

## Get test dataset

In [4]:
RL_DATA_PATH = os.getenv("RL_DATA_PATH")
df = pd.read_csv(RL_DATA_PATH + "/test_random.csv", sep=';')

## Generate Model responses

In [5]:
# Function to generate response
def generate_response(prompt, tokenizer, model, max_length=1024):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_length = inputs['input_ids'].shape[1]
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=input_length + max_length, do_sample=True, top_k=50)
        generated_ids = outputs[0][input_length:]
    return tokenizer.decode(generated_ids, skip_special_tokens=True)

In [6]:
# Apply both models
df['response_base_model'] = df['prompt'].apply(lambda x: generate_response(x, tokenizer, base_model))
df['response_new_model'] = df['prompt'].apply(lambda x: generate_response(x, tokenizer, new_model))

# Show result
print(df[['prompt', 'response_new_model', 'response_base_model']])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


                                              prompt  \
0  \n\n\n                --- Definitie ---\n\n   ...   
1  \n\n\n                --- Definitie ---\n\n   ...   
2  \n\n\n                --- Definitie ---\n\n   ...   
3  \n\n\n                --- Definitie ---\n\n   ...   

                                  response_new_model  \
0   Inhoud: In elk geval het budgettaire totaalbe...   
1  <inhoud van de preconditie>\n\n               ...   
2  ----------------------------------------------...   
3  1. Voorwaarde voor het weigeren recht op algem...   

                                 response_base_model  
0  -------\n\n                Subfact: Begrotings...  
1  1. De vreemdeling beschikt over een geldige ma...  
2                 1. Een preconditie voor de act ...  
3  ------------------------------\n\n\n          ...  


In [7]:
# store response df in csv
df.to_csv(EVAL_ANSWERS_CSV, index=False, sep=';')

'prompt;precondition_texts;precondition_positions;response_base_model;response_new_model\n"\n\n\n                --- Definitie ---\n\n                Preconditie: Een preconditie beschrijft de omstandigheden waaronder de handeling wettelijk kan worden uitgevoerd.\n                Act: Een act kan worden uitgevoerd door een agent binnen het normatieve systeem dat wordt gedefinieerd door het juridische document.\n                Fact: Fact frames beschrijven zaken waarvan de aanwezigheid of waarheidswaarde de toestand van het normatieve systeem kenmerkt. \n\n                \n\n                \n\n                                --- Gedachteketen ---\n\n                                1. Zoek alle vermeldingen van de fact in de tekst.\n                                2. Zoek in de artikelen waarin de fact wordt genoemd naar specifieke subfacts voor de fact. \n                                3. Zoek ook naar specifieke verwijzingen naar andere artikelen waarin mogelijk andere subfacts voo

In [8]:
print(df['response_base_model'][0])
print(df['response_new_model'][0])

-------

                Subfact: Begrotingsstaat, Budgettaire beschouwingen
 	                Positie: Artikel 4.4, derde lid, letter b IN Comptabiliteitswet 2016

                -------

                Subfact: Budgettaire totaalbeeld
 	                Positie: Artikel 4.4, derde lid, letter a IN Comptabiliteitswet 2016

                -------

                Subfact: Overzicht van de uitgaven en de ontvangsten
 	                Positie: Artikel 4.4, derde lid, letter c IN Comptabiliteitswet 2016
 Inhoud: In elk geval het budgettaire totaalbeeld, de budgettaire beschouwingen en een overzicht van de uitgaven en ontvangsten.  

                Hoofdstuk 2. Inrichting van de begroting  

                Artikel 2.23. Indienen van de rijksbegroting  

                  Subfact: Budgettaire totaalbeeld en budgettaire beschouwingen

                Positie: 4a en 4b in artikel 2.23 van de Comptabiliteitswet 2016

                  Subfact: Overzicht van de uitgaven en ontvangsten

    

In [27]:
candidates_new = df['response_new_model'].tolist()
candidates_base = df['response_base_model'].tolist()

precon_text_list = df['precondition_texts'].to_list()
precon_pos_list = df["precondition_positions"].to_list()

references = []
for dict1, dict2 in zip(precon_text_list, precon_pos_list):
    dict1 = ast.literal_eval(dict1)
    dict2 = ast.literal_eval(dict2)
    combined = []
    for key in dict1.keys():  # or use sorted(dict1.keys()) if key order isn't guaranteed
        combined.append(str(dict1[key]) + '\n')
        combined.append(str(dict2[key]) + '\n\n')
    references.append(''.join(combined))

print(references)

['een overzicht van de uitgaven en de ontvangsten in de begrotingen voor het begrotingsjaar en de vier daarop aansluitende jaren.\nArtikel 2.23 sectie 4c IN Comptabiliteitswet 2016\n\nde budgettaire beschouwingen over het voorgenomen beleid voor de collectieve sector\nArtikel 2.23 sectie 4b IN Comptabiliteitswet 2016\n\nhet budgettaire totaalbeeld voor het betrokken begrotingsjaar en de vier daaropvolgende jaren van de rijksbegroting en de niet tot de rijksbegroting behorende budgetdisciplinesectoren\nArtikel 2.23 sectie 4a IN Comptabiliteitswet 2016\n\n', 'de verblijfsvergunning wordt verleend met ingang van de dag waarop de vreemdeling heeft aangetoond dat hij aan alle voorwaarden voldoet\nArtikel 26, sectie 1 IN Vreemdelingenwet 2024\n\nde verblijfsvergunning wordt verleend met ingang van een dag eerder dan de dag waarop de aanvraag is ontvangen\nArtikel 26, sectie 1 IN Vreemdelingenwet 2024\n\n', 'Begroting bevat begrotingsstaat\nArtikel 2.2 IN Comptabiliteitswet 2016\n\n', 'colleg

## ROUGE/BLEU on relevant sequence from the answer 

In [34]:
# Evaluation metric 1: ROUGE on relevant sequences
# WHy not BLEU --> penalizes missing ngrams, not something I m looking for here

rouge = evaluate.load('rouge')

results_new = rouge.compute(predictions=candidates_new, references=references)
results_base = rouge.compute(predictions=candidates_base, references=references)

print(f"Results new: {results_new}")
print(f"Results base: {results_base}")




Results new: {'rouge1': np.float64(0.32155369137579465), 'rouge2': np.float64(0.14458924733614864), 'rougeL': np.float64(0.23140773059853664), 'rougeLsum': np.float64(0.30144503918275956)}
Results base: {'rouge1': np.float64(0.22510808879830946), 'rouge2': np.float64(0.09184750625230363), 'rougeL': np.float64(0.16605545339128516), 'rougeLsum': np.float64(0.22174885260463903)}


## BERT Score --> based on embedding similarity

In [28]:
# Maybe add BERTScore --> semantic similarity based on sentencetransformer
P_new, R_new, F1_new = score(
    candidates_new, 
    references, 
    model_type='answerdotai/ModernBERT-base', 
    num_layers=22,
    lang='nl')

P_base, R_base, F1_base = score(
    candidates_base, 
    references, 
    model_type='answerdotai/ModernBERT-base', 
    num_layers=22,
    lang='nl')

In [31]:
print(f"BERT Score metrics new: {P_new, R_new, F1_new}")
print(f"BERT Score metrics base: {P_base, R_base, F1_base}")

print(f"F1 base: {F1_base.mean()}, F1 new: {F1_new.mean()}")



BERT Score metrics new: (tensor([0.8441, 0.7226, 0.7086, 0.8301]), tensor([0.8497, 0.8189, 0.8137, 0.8155]), tensor([0.8469, 0.7678, 0.7575, 0.8227]))
BERT Score metrics base: (tensor([0.8104, 0.7440, 0.6925, 0.7777]), tensor([0.8275, 0.8283, 0.8064, 0.8154]), tensor([0.8189, 0.7839, 0.7451, 0.7961]))
F1 base: 0.7859886884689331, F1 new: 0.7987325191497803


## G-Eval

In [ ]:
# Evaluation metric 2: Evaluate whole answer on G-Eval

## Personal evaluation

In [ ]:
# Generate about 10 answers with both models --> load them into the user interface and get reward score
# Also maybe do qualitative evaluation